# Datasets

A [Dataset](https://nexus-forge.readthedocs.io/en/latest/interaction.html#dataset) is a specialization of a [Resource](https://nexus-forge.readthedocs.io/en/latest/interaction.html#resource) that aims to register (upload) files with eventually metadata in a configured store.

In [ ]:
from kgforge.core import KnowledgeGraphForge

A configuration file is needed in order to create a KnowledgeGraphForge session. A configuration can be generated using the notebook [00-Initialization.ipynb](00%20-%20Initialization.ipynb).

Note: DemoStore doesn't implement file operations yet. Use the BluBrainNexus store instead when creating a config file.

In [ ]:
forge = KnowledgeGraphForge("../../configurations/forge.yml")

## Imports

In [ ]:
from kgforge.core import Resource

In [ ]:
from kgforge.specializations.resources import Dataset

In [ ]:
import pandas as pd

## Creation with files

In [ ]:
! ls -p ../../data | egrep -v /$

In [ ]:
persons = Dataset(forge, name="Interesting Persons")

In [ ]:
persons.add_files("../../data/persons.csv")

In [ ]:
forge.register(persons)

In [ ]:
forge.as_json(persons)

In [ ]:
associations = Dataset(forge, name="Associations data")

In [ ]:
associations.add_files("../../data/associations.tsv")

In [ ]:
associations.add_derivation(persons)

In [ ]:
forge.register(associations)

In [ ]:
forge.as_json(associations)

In [ ]:
# The argument overwrite: bool can be provided to decide whether to overwrite (True) existing files with the same name or
# to create new ones (False) with their names suffixed with a timestamp  
associations.download(path="./downloaded/", source="parts")

In [ ]:
! ls -l ./downloaded

In [ ]:
# ! rm -R ./downloaded/

## Creation with resources

In [ ]:
distribution_1 = forge.attach("../../data/associations.tsv")

In [ ]:
distribution_2 = forge.attach("../../data/persons.csv")

In [ ]:
jane = Resource(type="Person", name="Jane Doe", distribution=distribution_1)

In [ ]:
john = Resource(type="Person", name="John Smith", distribution=distribution_2)

In [ ]:
persons = [jane, john]

In [ ]:
forge.register(persons)

In [ ]:
dataset = Dataset(forge, name="Interesting people")

In [ ]:
dataset.add_parts(persons)

In [ ]:
forge.register(dataset)

In [ ]:
forge.as_json(dataset)

In [ ]:
dataset.download(path="./downloaded/", source="parts")

In [ ]:
! ls -l ./downloaded

In [ ]:
# ! rm -R ./downloaded/

## Creation from a dataframe

See notebook `DataFrame IO.ipynb` for details on conversions of instances of Resource from a Pandas DataFrame.

### basics

In [ ]:
dataframe = pd.read_csv("../../data/persons.csv")

In [ ]:
dataframe

In [ ]:
persons = forge.from_dataframe(dataframe)

In [ ]:
forge.register(persons)

In [ ]:
dataset = Dataset(forge, name="Interesting people")

In [ ]:
dataset.add_parts(persons)

In [ ]:
forge.register(dataset)

In [ ]:
forge.as_json(dataset)

### advanced

In [ ]:
dataframe = pd.read_csv("../../data/associations.tsv", sep="\t")

In [ ]:
dataframe

In [ ]:
dataframe["distribution"] = dataframe["distribution"].map(lambda x: forge.attach(x))

In [ ]:
associations = forge.from_dataframe(dataframe, na="(missing)", nesting="__")

In [ ]:
forge.register(associations)

In [ ]:
dataset = Dataset(forge, name="Interesting associations")

In [ ]:
dataset.add_parts(associations)

In [ ]:
forge.register(dataset)

In [ ]:
forge.as_json(dataset)